In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from pandas import DataFrame

url = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv"
re = requests.get(url)
re_text = re.text
file_name = url.split("/")[-1]
f = open(file_name, 'w')
f.write(re_text)
f.close()

In [20]:
import pandas as pd
from datetime import date

def impute_province_state(df, impute_col, replace_col, insert_index):
    imputed_province_state = [country if pd.isna(province) else province for province, country in zip(df[impute_col].values, df[replace_col].values)]
    df = df.drop(impute_col, axis=1)
    df.insert(insert_index, impute_col, imputed_province_state)
    return df

def get_covid_19_ts():
    data_names = ['confirmed', 'deaths']
    covid_19_ts = pd.DataFrame()
    for dn in data_names:
        csv_url = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_{}_global.csv".format(dn)
        ts_df = pd.read_csv(csv_url)
        ts_df.insert(0, 'Case', dn)
        covid_19_ts = covid_19_ts.append(ts_df)
        print("Appending {}...".format(dn))
    covid_19_ts = impute_province_state(covid_19_ts, 'Province/State', 'Country/Region', 1)
    covid_19_ts.to_csv('time_series_covid19.csv', index=False)
    return covid_19_ts

def get_daily_report(covid_ts_df):
    last_column = covid_ts_df.columns[-1]
    month, day, year = last_column.split("/")
    month = int(month)
    day = int(day)
    year = int(year) + 2000
    latest_date = date(year, month, day)
    latest_date_str = latest_date.strftime('%m-%d-%Y')
    csv_url = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/{}.csv".format(latest_date_str)
    daily_report = pd.read_csv(csv_url)
    daily_report = impute_province_state(daily_report, 'Province_State', 'Country_Region', 2)
    daily_report.to_csv('daily_report_covid19.csv', index=False)
    return daily_report

In [3]:
import pandas as pd
from pandas import DataFrame
path = "C:/Users/ASUS/Desktop"
filename = path + "武漢肺炎.csv"
df = pd.read_csv(filename, 0)
df["Long"]

TypeError: object of type 'int' has no len()